# Language models
A language model is an algorithm that takes a sequence of words, and outputs the likely next word in the sequence. Most language models output a list of words, each with its probability of occurance. For example, if we had a sentence that started `I would like to eat a hot`, then ideally the algorithm would predict that  the word `dog` had a much higher chance of being the next word than the word `meeting`. Many of you may be familiar with some aspects of this concept from CS136's WordGen lab, or CS134's Generators lab assignment, although, in that case you were likely using characters rather than entire words in your language models.

Language models are a very powerful building block in natural language processing. They are used for classifying text (e.g. is this review positive or negative?), for answering questions based on text (e.g. "what is the capital of Finland?" based on the Wikipedia page on Finland), and language translation (e.g. English to Japanese).

## The intuition behind why language models are so broadly useful
How can this simple sounding algorithm be that broadly useful? Intuitively, this is because predicting the next word in a sentence requires a lot of information, not just about grammar and syntax, but also about semantics: what things mean in the real-world. For instance, we know that `I would like to eat a hot dog` is semantically reasonable, but `I would like to eat a hot cat` is nonsensical. 

I trained a simple language model, and asked it to predict the word following `I would like to eat a `. We get: `hamburger`

The rest of this notebook will describe how to set up a language model using python modules available to us, to make such word predictions as well as classifications. We will use these modules to classify and generate text in an automated chatbot on Twitter and then reflect on the process, bringing together all our class discussions about conversational agents, natural language technologies, and human-in-the-loop systems!
    

# Step 0: Set-up for this Assignment

You'll need to install some new modules to complete this assignment. Set-up instructions are always available via the project README in the Github repository.

Once you've done so, you should be able to run the following code:

In [ ]:
# Follow install instructions in the README, prior to running this notebook!

# If using Google Colab, uncomment these two lines:
#!pip install fastai==2.0.19
#!pip install fastcore==1.3.2

from fastai.text import *
import pandas as pd

# Step 1: Load all the data 
In this activity, we are going to use a dataset of tweets from the satirical news site, [The Onion](https://www.theonion.com), as well as some non-sarcastic news sources. The data set is from [Kaggle](https://www.kaggle.com/rmisra/news-headlines-dataset-for-sarcasm-detection). 

The dataset is available as a JSON file in the included folder, `lib`.


In [ ]:
from pathlib import Path
data_path = Path('./lib')

The data is in a JSON file, so we are using the `read_json` method to read-in the data. If you have different data that is CSV, use the `read_csv` method instead. 

We use the `lines=True` argument here because the author formatted each line as a separate JSON object. At least half of your time as a data scientist/AI researcher is spent dealing with other people's data formats!


In [ ]:
headlines = pd.read_json(data_path/'Sarcasm_Headlines_Dataset_v2.json', lines=True)

In [ ]:
headlines

As you can see, some of this dataset is drawn from _The Onion_, the rest is drawn from places like the Huffington Post which publishes real news, not satire. 

## Step 1a: Examine the data set (5% effort)

Before we go off adventuring, let's first see what this dataset looks like. 

### Q1.1: How large is this dataset? Is it balanced? (1% effort)

**ANSWER:**  _Double click this text to write your answer to the question here._

In [4]:
# Insert code here to check size of dataset, and how many are positive (is_sarcastic = 1) and how many negative?
# Hint: Your output will look like this:


0    14985
1    13634
Name: is_sarcastic, dtype: int64

### Q1.2: How many words on average is each headline? (4% effort)
Longer text = more information. We want to see what the length of the headline is in order to see how much information it may have. 

_Hint:_ See https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.count.html (the the `\s` regex looks for spaces).

**ANSWER:** _Double click this text to write your answer to the question here._

In [ ]:
# Insert code here to find the average length of headline (in words)



# Step 2: Build a language model that knows how to write news headlines

This is the first step of our project that will be using a machine learning model. 

We are going to use the [fast.ai](https://fast.ai/) library to create this model. If you need help with understanding this section, look at the fast.ai documentation -- it is fantastic! The steps below are modified from the [online tutorial](https://docs.fast.ai/tutorial.text.html#The-ULMFiT-approach).

In [ ]:
import fastai
from fastai.text.all import *

*Note: if this import fails for you, make sure you've installed fastai first. This is not super straight-forward. Close the notebook and follow the instructions in the README.*

In [ ]:
dls = TextDataLoaders.from_df(headlines, path=data_path, text_col='headline', is_lm=True, valid_pct=0)
print("This code takes ~1 minute to run on Iris' laptop, and possibly more than 10 minutes on Colab.")


First, we tell fastai that we want to work on a list of texts (headlines in our case), that are stored in a dataframe (that's the `TextDataLoaders.from_df` part.) We also pass in our data path, so after we process our data, we can store it at that location. We tell it where to look for the headline in the dataframe (which  column to use, `text_col=`). We specify that it is a language model (`is_lm`) so it labels the "next word" as the label for each sequence of words. But what kind of validation data do we need for a language model? Remember that a language model predicts the next word in an input sequence of words. So, we can't just take some of the headlines and set them aside as validation data. Instead, we want to use all the sentences and validate whether we can guess the right next word some fraction of the time. So, we tell it not to split the data into validation sets (`valid_pct=0`). 

## Step 2a: Learn the model

Now that we have the data, it's time to train the model.

Now, we *could* learn a language model from scratch. But we're instead going to cheat. We're going to use a pretrained language model, and finetune it for our purpose. Specifically, we're going to use a model trained on the `Wikitext-103` corpus. 

One way to understand it is to think of our pre-trained model is as a model that can predict the next word in a Wikipedia article. We want to train it to write headlines instead. Since headlines still have to sound like English, ie. follow grammar, syntax, be generally plausible etc, being able to predict the next word in Wikipedia is super useful. It allows us to start with a model that already knows some English, and then just train it for writing headlines.



In [ ]:
learn = language_model_learner(dls, AWD_LSTM, drop_mult=0.5)

This `AWD_LSTM` is the pretrained Wikipedia model.

Let's train it.

In [ ]:
learn.fit_one_cycle(1, 1e-2)
print("This code takes Iris' laptop about 8 minutes to run, and around 15 minutes on Colab.")

Once trained, it's time to write some headlines! We give it a starting sequence `Students protest ` and see what it comes up with. 

_Note:_ Don't worry about a `UserWarning: Your generator is empty.` warning, unless the next two lines of code don't work!

_Note:_ If you were unable to get the model training/building working, it _might_ be possible to load the pre-built `headlines-lm.pkl` model that's available [via Google Drive](https://drive.google.com/drive/folders/1aKO1eWeGJmHZFrslRnMQRV1PVFiPv6eB?usp=sharing) and load it with `load_learner(fname='lib/headlines-lm.pkl')`. If everything is working, don't worry about this comment!

In [ ]:
learn.predict("Students protest ", n_words=5, no_unk=True)

Pretty good, huh? 

In [ ]:
learn.predict('The Fed is expected to', n_words=3, no_unk=True)

OK, it's not perfect! Let's make it a little better. 

The `unfreeze` below is telling fastai to allow us to change the weights throughout the model. We do this when we want to make the model generate text that's more similar to our headlines (than to Wikipedia). 

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(1, 1e-3)
print("This code takes Iris' laptop about 11 minutes to run, 20 minutes on Colab.")

In [ ]:
learn.predict('New Study', n_words=5)

In [ ]:
learn.predict('16 Problems', n_words=5)

OK, now let's save our hard work. We'll use this later. (Pssst: why is it called an encoder? Look at the Fastai docs to find out!)

In [ ]:
learn.save_encoder('headlines-awd.pkl')

Note that we also want to save the whole model, so we can reuse it in our twitter bot. 


In [ ]:
learn.export('headlines-lm.pkl')

# Step 2b: See how well the language model works (15% effort)

Try generating a few more headlines. Then, answer the following questions. Wherever possible, show what code you ran, or what predictions you asked it for. 

*Suggestion: Try using punctuations, numbers, texts of different lengths etc.*

### Q2.1: What is the effect of starting with longer strings? (5% effprt)

We could start our headline generation with just one word, e.g. `learn.predict('White', n_words=9)` or with many: `learn.predict('White House Says Whistleblower Did', n_words=5)`. What is the difference you see in the kinds of headlines generated?

**ANSWER:** _Double click this text to write your answer to the question here._

In [ ]:
learn.predict('White', n_words=9)
#learn.predict('White House Says Whistleblower Did', n_words=5)


### Q2.2: What aspects of the task of generating headlines does our language model do well? (5% effort)
For example, does it get grammar right? Does it know genders of people or objects? etc.

**ANSWER:** _Double click this text to write your answer to the question here._

In [ ]:
# Optional code block.






### Q2.3: What aspects of the task of generating headlines does our model do poorly? (5% effort)
What does it frequently get wrong? Why might it make these mistakes?

**ANSWER:** _Double click this text to write your answer to the question here._

In [ ]:
# Optional code block.





# Step 3: Learn a classifier to see which headlines are satire

Remember, our dataset has some stories that are satire (from _The Onion_) and others that are real. Now, we're going to train a classifier to distinguish one from the other. 

In [ ]:
dls_clas = TextDataLoaders.from_df(headlines, path=data_path, text_col='headline', label_col='is_sarcastic', valid_pct=0.2, text_vocab=dls.vocab)
print("This code takes Iris' laptop about 1 minute to run, 13 minutes in Colab.")

We're using a similar DataLoaders method as we did for our language model above. Here, we specify the target column with `label_col` and we use `valid_pct=0.2` so we keep some fraction of our dataset as a validation set. There is one other trick: `text_vocab=dls.vocab` ensures that our classifier only uses words that we have in our language model -- so it never deals with words it hasn't encountered before. (Consider: why is this important?)

See if you can work out what the other arguments are. 

In [ ]:
dls_clas.show_batch()

Above: what our data looks like after we apply the vocabulary restriction. `xxunk` is an unknown word. `xxpad` is a padding character used to ensure all strings are the same length. 

Below: we're creating a classifier:

In [ ]:
classify = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

Remember that language model we saved earlier? It's time load it back!

In [ ]:
classify.load_encoder('headlines-awd.pkl')

What's happening here? 

Here's the trick: a language model predicts the next word in a sequence using all the information it has so far (all the previous words). When we train a classifier, we ask it to predict the label (satire or not) instead of the next word. 

The intuition here is that if you can tell what the next word in a sentence is, you can tell if it is satirical. (Similarly, if you can can tell what the next word in an email is, you can tell if it is spam, etc.)

In [ ]:
classify.fit_one_cycle(1, 1e-2)
print("This line takes Iris' laptop about 3 minutes to run, Colab 6 minutes.")

In [ ]:
classify.freeze_to(-2)

Above: this is similar to `unfreeze()` that we used before. Except, you only allow a few layers of your model to change. Then we can train again, similar to using `unfreeze()`

In [ ]:
classify.fit_one_cycle(1, 1e-3)
print("This line takes Iris' laptop about 5 minutes to run, Colab 7 minutes.")

Wow! An accuracy of >85%! (_Consider: Where is accuracy reported?_) That sounds great, and for not that much work. 

Now, let's try it on some headlines, to see how well it does. 

# Step 4: Try out the classifier (20% effort)

In [ ]:
classify.predict("Despair for Many and Silver Linings for Some in California Wildfires")

Here in the output, the first part of this tuple is the chosen category (`0`, i.e. not satire). The second part of the result is the index of "0" in our data vocabulary, and the last part is the probabilities attributed to each class (98.1% for `0` and 0.02% for `1`). The classifier suggests that the headline (which I got from the [New York Times](https://www.nytimes.com/2019/10/29/us/california-fires-homes.html?action=click&module=Top%20Stories&pgtype=Homepage)) is not satire and it seems pretty confident that's the case (98.1% probability). 

## Step 4a: Try out this classifier (10% effort)

Below, try the classifier with some headlines, real or made up (including made up by the language model above). 


### Q4.1: Two headlines that the classifier correctly classifies 

**ANSWER:** 

1. _Double click this text to write your answer to the question here._
2. _Double click this text to write your answer to the question here._

### Q4.2: Two headlines that the classifier classifies incorrectly 

**ANSWER:** 

1. _Double click this text to write your answer to the question here._
2. _Double click this text to write your answer to the question here._

In [ ]:
# Cell Block for showing your work with attempted headlines


Now, we want to find two headlines that the classifier is really confident about, but classifies incorrectly. We want the confidence of the prediction to be at least 85%.

One headline is anything you want to write. Another must be a real headline (not satire) that you could trick the classifier into misclassifying changing only one word. For instance, taking `"Despair for Many and Silver Linings for Some in California Wildfires"`, a real NYTimes headline, you can change it to `"Despair for Many and Silver Linings for Some in Oregon Wildfires"` (note that this particular change does not cause the classifier to misclassify).

### Q4.3: One headline that the classifier classifies incorrectly, with false high confidence. (4% effort)

**ANSWER:** _Double click this text to write your answer to the question here. How is it misclassified? What is the confidence?_

### Q4.4: One real headline, with one word changed, that the classifier classifies as satire, with false high confidence. (4% effort)

**ANSWER:** _Double click this text to insert link to the original headline/article._

**ANSWER:** _Double click this text to record the confidence._

**ANSWER:** _Double click this text to write your answer to the question here._


## Step 4b: What kinds of headlines are misclassified? (10% effort)

### Q4.5: Write your hypothesis below on what kinds of headlines are misclassified. 
Show your work. (fastai v1 used to have a `TextClassificationInterpretation` utility that doesn't appear to exist anymore, but might've been helpful for this step!). 

**ANSWER:** _Double click this text to write your answer to the question here._

In [2]:
# Show your work here


# Step 5: Save your classifier
Now that we've trained the classifier, you're ready for Part 2. You'll use this saved file in your bot later.

In [ ]:
classify.export(fname='satire_awd.pkl')

Later, you'll use it like so.

In [ ]:
serve_classifier = load_learner(fname=str(data_path)+'/satire_awd.pkl')
serve_lm = load_learner(fname=str(data_path)+'/headlines-lm.pkl')

In [ ]:
serve_classifier.predict('How the New Syria Took Shape')

In [ ]:
serve_lm.predict('Rising Seas', n_words=7)

# Step 6: Set-up your Twitter Bot (20% effort)

Follow the instructions in Glow to set-up your Twitter Developer account!

### Q6.1: Reflect on the sign-up process (5% effort) 
As you go through the process of signing up for a twitter developer account, reflect on the questions you are asked, why you are being asked them, and how you think they will serve the intended purpose (or not). 

**ANSWER:** _Double click this text to write your answer to the question here._


## Important! Tweet Rate Limit!
It's possible that Twitter will rate limit you developer account, try to reply/post sparingly during your testing phase!

## (6.2) Test whether your script has access to Twitter
I'm assuming you've stored your Twitter keys in `credentials.py` saved in the same directory as this notebook, and the file looks like the following format:
```
consumer_key = 'XXX' # API key
consumer_secret = 'XXX' # API secret
access_token = 'XXX' # Access token
access_token_secret = 'XXX' # Access secret
```

This snippet of code should access your Twitter account, fetch your name stored there, and then print it out. If it doesn’t, go back and debug!

In [ ]:
from credentials import *
import tweepy

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# The internet suggests that 'API.get_user' or 'API.verify_credentials' 
# may be the replacement for api.me(): https://stackoverflow.com/questions/69689678/tweepy-error-api-object-has-no-attribute-me
# However, if neither of these work, that's okay! We just need to test to see if we successfully connected to our bot!
#user = api.me() # deprecated
user = api.get_user(ENTER_USER_ID_HERE)
print (user.name)

The next step is to find a tweet that mentioned your bot (starting with the `@` symbol), which are the messages that your bot will respond to. You'll need to tweet yoru bot for this to work! Here’s some template code for searching for tweets with Tweepy for a hypothetical bot, `@satirebot`, you'll need to modify it to fit your needs:

In [ ]:
####
# Define the search
#####
query = '@satirebot' # replace with your bot's twitter name!
max_tweets = 100

####
# Do the search
#####
searched_tweets = []
last_id = -1

while len(searched_tweets) < max_tweets:
    count = max_tweets - len(searched_tweets)
    try:
        new_tweets = api.search(q=query, count=count, max_id=str(last_id - 1))
        if not new_tweets:
            break
        searched_tweets.extend(new_tweets)
        last_id = new_tweets[-1].id
    except tweepy.TweepError as e:
        # depending on TweepError.code, one may want to retry or wait
        # to keep things simple, we will give up on an error
        break
####
# Iterate over the search
#####
for status in searched_tweets:
    # do something with all these tweets
    print(status)

If you run this snippet, it will print out the messy JSON version of all the statuses returned. You can see what sorts of things each of the status objects contain. Some important ones are:

* `text` -- the text of the returned tweet
* `author.screen_name` - the Twitter username of the user who sent the tweet
* `id` - id of the tweet, which you can use to reply to it, or search only for tweets that were posted after it

In [ ]:
# Insert code in this cell to access the useful information from the JSON status:



And, of course, you want to be able to send new tweets too, which is done like this:

In [ ]:
api.update_status('I\'m in a plane!')

You will likely want to not just send a status, but actually reply to the original tweet. You can combine some of what you’ve learned here to do that (You have to include the @-mention in the tweet, or it won’t show up as a reply.):

In [ ]:
api.update_status(
  'this is a reply! @' + status.author.screen_name,
  status.id_str
)

### Q6.2: Adapt the code from above to respond to a query, as described above. (15% effort)

Copy the code from above and modify it to search for Tweets referencing your bot (i.e., Tweets containing `@YourBotName`), and then reply to those tweets, perhaps with something using the tweet's text such as the number of characters in the original tweet.

Of course, this is just the tip of the iceberg with what you can do with Tweepy. If you’re interested, there is a lot of documentation online. You might start with the [official Tweepy docs](http://docs.tweepy.org/en/v3.6.0/getting_started.html).

In [ ]:
# Insert code here

# Step 7: Integrate the bot with the satire classifier (40% effort)
Now that you can do basic replies with your bot, it’s time to make it do something useful! Specifically, our bot should do two things: 

1. When someone tweets a headline @ the bot, it replies with whether the headline is satire. 
2. It also makes up a headline that plays off the original headline, and tweets it back. 

Here’s an example (assuming the bot’s called @satirebot):

**User: @satirebot Rising Seas Will Erase More Cities by 2050, New Research Shows**

**satirebot: Yeah, looks real, not satire. But here’s what I say: “Rising Seas will sound great on national security”**


_Hint_: you want to call `api.update_status` using the outputs of our models (classifier and language models) instead of responding `this is a reply!`

The best assignments will have bots that:

- Respond with prediction of whether the given headline is satire or not (10% effort)
- Respond with a newly generated headline inspired by the original (10% effort)
- Interactions are designed to be friendly, non-misleading, and should let users to discover what is going on. (10% effort)

In [ ]:
# Feel free to add more cell blocks, as needed



### Q7.1: Reflect: Would you recommend using our satire-classifier as a good starting point to build a fake-news classifier? (10% effort)
If so, what changes would we need to make to make it useful for this purpose? If not, why not?

**ANSWER:** _Double click this text to write your answer to the question here._

# Extra Credit 1: Test with Users and Iterate (5% additional effort)
In this part, you’ll ask three participants to interact with your bot. You’ll give the user high-level information about what the domain of the bot is, and then see how they interact with it. Ask each of the participants to ask your chatbot at least three different things. Record how they interact with your bot. After this participant input, update your bot to attempt to address how that participant interacted with your chatbot. 

### Q EC1.1: How did what your participants input compare to the ones you tested so far? How did participants react when the chatbot didn’t respond correctly, or responded with nonsense? (2.5% additional effort)

**ANSWER:** _Double click this text to write your answer to the question here._

### Q EC1.2: What change could you make in response to this feedback? (2.5% additional effort)

**ANSWER:** _Double click this text to write your answer to the question here._

# Extra Credit 2: Deploy bot (10% additional effort)
Take this Jupyter notebook and deploy it such that it runs once an hour, and responds to all messages sent to it  
([Tweepy StreamListener](https://docs.tweepy.org/en/v3.10.0/streaming_how_to.html) might be useful here). You do not have to worry about finding a server to deploy this independently, but it should be runnable from the cell below. Run the bot, and then force-stop it so that I can see the output.

### Q EC2.1: Twitter handle (of bot) to test for.
**ANSWER:** _Double click this text to write your answer to the question here._

In [ ]:
# Feel free to add more cell blocks, as needed

# Assignment Submission

Once you've completed all of the above, you're done with assignment 4! As always, clean up your code and ensure your entire Jupyter Notebook runs before submitting, Iris must be able to run your notebook on her machine.

Once you think everything is set, please change the filename of your notebook to `[yourunixID]_haii21[assignmentnumber].ipynb`, e.g., `ikh1_haii21a4.ipynb` and then .zip your Notebook and any additional files you used (this is likely just the Notebook file and your JSON data in the lib/ directory: **DON'T** include your gigantic .pkl language model files, and **DON'T** include your credentials.py!), and submit the `.zip` file on GLOW.